In [5]:

#IIMPORTING THE IMPORTANT DIRECTORIES:
import cv2
import numpy as np


#FUNCTION TO TAKE CARE OF SCREEN NOT CLOSING:
def empty(a):
    pass

#DIFFERENT COLORS CAN BE USED USING AN ARRAY, HERE I HAVE USED ONLY 1 COLOR BLUE:
colArr = [[88, 125, 0, 126, 222, 255]]

#THESE POINTS WILL BE RECURRINGLY PRINTED:
points = []

#FUNCTION TO FIND COLOR:
def findColor(img, colArr):
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV);
    newPoints = []
    count=0;
    lower = np.array(colArr[0][0:3]);
    upper = np.array(colArr[0][3:6]);
    mask = cv2.inRange(imgHSV, lower, upper) 
#     cv2.imshow("img", mask)
    x,y = getContour(mask) #MASK WILL BE PASSED AS ARGUMENT TO GETCONTOURS, 
    cv2.circle(imgFinal, (x,y), 10, (255,0,0), cv2.FILLED)
    if x!=0 and y!=0:
        newPoints.append([x,y,count])
    count+=1
    return newPoints


#MAKES A BOX AROUND THE PEN CAP (THE BLUE ITEM) AND RETURNS THE CENTEE OF THE UPPER EDGE:
def getContour(img):
    contours,hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    x,y,w,h = 0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt) #Got the area of the contour
#         print(area)
        if area>500: #Minumum threshold for noise is important for general applications
            cv2.drawContours(imgFinal, cnt, -1, (255, 0, 0), 3)
            peri = cv2.arcLength(cnt,True) #Getting curve length (True for closed arcs)
            #print(peri)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True) #Approximate number of corners (0.02 is the resolution)
            x, y, w, h = cv2.boundingRect(approx) #Bounding box around the object
    return x+w//2,y 
 
#DRAWS A CIRCLE AROUND ALL POINTS:
def draw(points):
    for point in points:
        cv2.circle(imgFinal, (point[0], point[1]), 10, (255,0,0), cv2.FILLED)
    
W = 640 #FIXING FRAME WIDTH AND HEIGHT
H = 480

cap = cv2.VideoCapture(0)  #Captures video
cap.set(3, W)
cap.set(4, H)
cap.set(10, 150)

while True: #SCREEN WILL RETURN UNTIL YOU PRESS EXIT KEY
    success, img = cap.read()
    imgFinal = img.copy()
    newPoints = findColor(img, colArr) #Finds color of points and 
    if len(newPoints)!=0:
        for x in newPoints:
            points.append(x) #Adds the newPoint to the points
    if len(points)!=0:
        draw(points) #Draws all points
    cv2.imshow("Result", imgFinal) #Shows final image only, we can also print the mask and contours if we want.
    k = cv2.waitKey(30) & 0xff #Press Escape Key to stop running the program.
    if k==27:
        break
cap.release()
cv2.destroyAllWindows()